# Parameter Estimation and Time-Series Analysis

This notebook aims to provide an overview of my efforts to develop better optimization routines for time-series analysis of microbial community abundance data. The notebook includes a general overview of the project along with key results.

__Hypothesis__: being "smart" about optimization choices can lead to better parameter estimates using real ecological datasets, relative to using "off-the-shelf" optimization routines built into Python and/or R.

## Background and Motivation

Time-series are all over microbial ecology. Unfortunately, real data are messy and often don't conform to the requirements of parameter estimation software. Some examples include:

* Unevenly spaced time-points, when parameter estimation routines assume data are evenly spaced

* Missing environmental data, when parameter estimation routines assume environmental data were measured along with microbial observations.

* Seasonal time-series, or time-series with other long gaps, when parameter estimation routines assume data were recorded without long gaps.

Examples of ecological datasets subject to one or more of these problems include:

* NTL-MO - multiple years of data, but even spacing and long gaps (courtesy of long Wisconsin winters)

* Marine time-series (HOT, SPOT, BATS) - multiple years of data, but even spacing

* Human microbiome time-series - research this, try to find some that are disease-related. Schloss C. diff datasets?

## Proposed Workflow

I propose to develop a custom optimization routine tailored to multi-year ecological data series with unevenly-spaced time-points and long gaps. The routine will estimate microbe-microbe and microbe-environment ("ecological") parameters from data, and I will (hopefully) demonstrate my routine performs better than off-the-shelf software.

My routine will have two features:

1. Rather than performing interpolation before the parameter estimation, I will perform interpolation simultaneously with the estimation. This allows the values of missing data points to emerge naturally from the model rather than assuming a fixed structure for the missing data (linear interpolation, etc). This also has the advantage of capturing dynamics that don't conform to the standard interpolation functions (such as blooms and other non-monotonic behavior).

2. For datasets with large gaps (such as winter), rather than interpolating across the long gaps, I will perform a single optimization over multiple time-series (of one year each) in parallel, rather than over a single time-series spanning the entire length of the dataset.

The performance of my routine will be benchmarked against routines which perform interpolation across a single time-series spanning the length of the dataset.

Initial work will be performed on the __Trout Bog epilimnion 16S rRNA time series__, which currently has two years of replicated data (2007, 2008) and six more years being sequenced (2009, 2012-2016).

| Lake / Layer | Year | Replicated Samples |
|--------------|------|--------------------|
| TBE          | 2007 | 38                 | 
| TBE          | 2008 | 26                 | 
| TBE          | 2009 | 14                 | 
| TBE          | 2012 | 14                 | 
| TBE          | 2013 | 11                 | 
| TBE          | 2014 | 8                  | 
| TBE          | 2015 | 10                 | 
| TBE          | 2016 | 9                  | 

However, I'm worried about the sparsity of the Trout Bog datasets. Samples are taken once or twice a week, meaning many of the data values still need to be interpolated. I'm worried that having too few observations relative to the length of the data won't provide enough information for the least-squares estimation. I'm considering starting with one of the dense human microbiome datasets and iteratively removing samples to see if performance degrades at some point.

## OTU Selection

I have chosen to use [unique sequence variants](http://www.nature.com/ismej/journal/vaop/ncurrent/full/ismej2017119a.html) (of 16s rRNA genes) as my definition of a microbial species. There are a number of algorithms to identify USVs, I chose to use [deblur](https://github.com/biocore/deblur) due to our previous work with Rob Knight's group. Because Alex was unsure of the exact parameters used for her paper, I decided to redownload and re-deblur the data. Bonus, now I have a pipeline for when the new samples come in. Deblurring resulted in __71,915 OTUs__.

Furthermore, Robin has observed that some of the full-length reads are low quality, so it's worth thinking about more extensive QC when new samples come in.

![Qual Scores of TB epi samples](images/tbe-qual-scores.png)

To reduce this number, I chose to retain OTUs that had an __average relative abundance__ (across two samples per time point) in some fraction of samples (__persistence__). OTUs below this threshold were lumped into an "other OTU". The plot below shows the number of fraction of sequences retained and total number of OTUs for a bunch of parameter values.

![Persistence and Abundance Filtering](images/filter-persist-abund.png)

Values of 0.001 relative abundance and 20% persistence resulted in __72 OTUs, 90% total reads, and 50% of sequences in all samples__. However, in some samples the "other OTU" contained almost 50% of the sequences from the sample. Upon inspection, this "other OTU" was dominated by three sequences with > 0.01 relative abundance.

Thus I added a third criterion to the filter: I chose to retain OTUs that had a __relative abundance__ in some fraction of samples (__persistence__) __OR__ were abundant in at least one sample at some relative abundance (__bloom__).

Sadly I have no 3D plot of these criteria. But criteria of 0.001 in at least 15% of samples OR an abundance of 0.01 in at least one sample resulted in 76 OTUs. Other parameter choices are possible, but these values retained ~90% of sequences in each sample while minimizing the number of total OTUs.

### Final Parameters
Rel. abund. and persistence - 0.001 relative abundance in at least 15% of samples  
Bloom - 0.01 relative abundance  
Total OTUs: 137

__Note__ for Cristina: Mean-variance scaling plots and Breusch–Pagan tests of heteroskedasticity suggest a relative abundance cutoff between 0.001 and 0.003 to be outside the realm of noise.

## Model Overview

I intend to develop a (possibly multivariate) [auto-regressive model](https://en.wikipedia.org/wiki/Autoregressive_model) (AR) to predict time-series data. I selected AR modeling due to its long use in ecology, and the fact that it is stochastic. Due to the long time periods between observations (days to weeks) and the short time scale of microbial behavior (hours to days), I don't believe that a mechanistic model makes much sense.

I will be following the [Box-Jenkins method](https://en.wikipedia.org/wiki/Box%E2%80%93Jenkins_method) to select the appropriate AR model. This method applies (a possibly simplified) autoregressive integrated moving average (ARIMA) model to find the best fit of model to a time-series.

ARIMA models contain three components. The auto-regressive (AR) term indicates that the variable is a function of its own prior values. The integrated (I) term indicates that the variables have been replaced with their differences (difference between current and previous values). This differencing may be necessary to make the time-series stationary (see below). The moving average (MA) term indicates that the error term is linear combination of errors from the current and previous times. ARIMA models are denoted __ARIMA(p, d, q)__, where __p__ is the number of time lags in the auto-regressive model, __d__ is the number of differences in the integrated model, and __q__ is the number of time lags in the moving average model.

ARIMA models can also be generalized as <a href='http://onlinelibrary.wiley.com/wol1/doi/10.1890/0012-9615(2003)073[0301:ECSAEI]2.0.CO;2/abstract'>multi-variate AR models</a>, in which the abundance of an OTU is not only a function its own (previous) abundance, but also of other variables (such as other OTUs or environmental variables).

In this study, I propose to develop and assess three distinct models for time-series abundance of OTUs:

* Univariate auto-regressive model - OTU only depends on its previous value(s)

* Multivariate auto-regressive model, only species-species interactions - OTU depends in its previous value(s) and previous value(s) of other OTUs

And possibly, 

* Multivariate auto-regressive model with species-species and species-environment interactions - OTU depends in its previous value(s) and previous value(s) of other OTUs and environmental data

* Multivariate auto-regressive model with species-environment interactions - OTU depends in its previous value(s) and previous value(s) of environmental data

In any case, the first step is to determine the appropriate values of __p__, __d__, and __q__ for the ARIMA model.

## Model Selection: Single OTU ARIMA Model

The first model I plan to develop is a univariate auto-regressive model, in which the abundance of an OTU depends only on its previous abundance(s) plus an error term.

### Time-Series Stationarity

The __I__ term in an ARIMA indicates the number of differencing terms required to make sure the time series is stationary. A time-series is [stationary](https://en.wikipedia.org/wiki/Stationary_process) means that the mean and variance don't change with time. A time-series which is non-stationary must be transformed to become stationary. The most common causes of non-stationarity is a trend in the mean, which can be caused by a [unit root](https://en.wikipedia.org/wiki/Unit_root) or a [deterministic trend](https://en.wikipedia.org/wiki/Trend_stationary). If a process has a unit root, it must be differenced to become stationariy. If a process has a trend, it must be de-trended. Thus, our first task is to determine if our time-series are stationary or not. If not, we must make them stationary.

Stationarity can be assessed many ways including histograms of abundance data for each OTU (should be normally distributed) and via statistical testing, such as the [Mann-Kendall test](http://www.statisticshowto.com/mann-kendall-trend-test/), the [ADF test](http://www.statisticshowto.com/adf-augmented-dickey-fuller-test/), and the [KPSS test](http://www.statisticshowto.com/kpss-test/).

These tests can be summarized:

| Test | Null Hypothesis | Rejection Indicates | 
|--------------|------|--------------------| 
| Mann-Kendall | No trend | Trend |
| ADF-Mean | Non-stationarity due to a unit root | Stationary | 
| ADF-Trend | Non-stationarity due to a trend | Trend-stationary | 
| KPSS-Mean | Stationary | Non-stationarity due to a unit root
| KPSS-Trend | Trend-stationary | Non-stationarity due to a unit root

#### Dealing with Zeros

By their nature, relative abundance data are non-stationary so they must be transformed. I opted for a natural logarithm transformation. Of course, zero values cannot be log-transformed. Based on mean-variance scaling, relative abundances below 0.001 are indistinguishable from noise. Thus, after filtering, I replaced all relative abundances below 0.001 with a small number (0.00065) before log-transforming the data. A value of 0.00065 was selected based on [empirical studies](http://onlinelibrary.wiley.com/doi/10.1002/9781119976462.ch4/summary) of dealing with zeros. 

#### Statistical Tests of Stationarity

Representative time-series and histograms of ln(relative abundance) are shown below. As you can see, the time-series do not appear to be stationary.

![Representative Log-transformed Time-Series](images/log-transformed-time-series.png)

Statistical testing confirms this. Because the statistical tests require evenly-spaced time-series, a perfomed the tests using a variety of interpolation methods. Results were generally consistent and are shown for the linear case. For all 152 time-series (76 OTUs x 2 years), the results of the statistical tests were:

| Test | Outcome | # Time-series
|--------------|------|--------------------| 
| Mann-Kendall | Reject null, detect a trend | 127
| ADF-Mean | Reject null, stationary | 7
| ADF-Trend | Reject null, trend-stationary | 9
| KPSS-Mean | Reject null, non-stationary due to a unit root | 91
| KPSS-Trend | Reject null, non-stationary due to a unit root | 73

Overall, these results suggest the data are non-stationary, and this may arise due a trend (M-K test) or a unit root (KPSS test).

I then took the first-difference of the time-series. For a time-series, the first difference of $Y_t$ is $Y_t - Y_{t-1}$. I performed first-differencing using a number of interpolation schemes. Plots of representative time-series and histograms of ln(relative abundance) are shown below. As you can see, the time-series appear to be stationary.

![Representative Log-transformed Time-Series](images/log-transformed-diff-time-series.png)

Statistical testing confirms this. Because the statistical tests require evenly-spaced time-series, a perfomed the tests using a variety of interpolation methods. Results were generally consistent and are shown for the linear case. For all 152 time-series (76 OTUs x 2 years), the results of the statistical tests were:

| Test | Outcome | # Time-series
|--------------|------|--------------------| 
| Mann-Kendall | Reject null, detect a trend | 55
| ADF-Mean | Reject null, stationary | 111
| ADF-Trend | Reject null, trend-stationary | 78
| KPSS-Mean | Reject null, non-stationary due to a unit root | 3
| KPSS-Trend | Reject null, non-stationary due to a unit root | 9

These results suggest that the first-differenced time-series are trend-stationary (non-stationary due to a deterministic trend in the mean). The next step is to detrend the time-series. Furthermore, the results indicate that the ARIMA process has __d__ = 1.

### Detrending

The statistical tests indicate many of the OTUs exhibit a trend in their abundance. This is borne out in plots of (log-transformed) relative abundance.

Some OTUs have trends...

![Time-Series with Trends](images/time-series-trends.png)

while others do not.

![Time-Series without Trends](images/time-series-no-trends.png)

I speculate that these trends are seasonal. When plotting two years of data (2007 and 2008), seasonality is observed for some taxa.

![Recurring Seasonal Trends](images/annual-trends.png)

Statistical tests again detect a trend, but with less confidence I suspect that this is because the time-series starts in spring of one season and ends during fall of another, so the seasonal trend is reflected in the data. Hopefully the support for a long-term trend will go away once additional years of data are added.

Because de-trending is intended to remove long-term trends in the data, not periodic variability, I have opted not to de-trend the data at this time.

### Auto-Regressive and Moving-Average Components

After a time-series has been made stationary, the next step is to determine the order of the auto-regressive and moving-average components. The [auto-regressive component](https://en.wikipedia.org/wiki/Autoregressive_model) of the model describes the dependency of the current value on its previous value(s), plus a stochastic term. In contrast, in a [moving-average model](https://en.wikipedia.org/wiki/Moving-average_model), the current value depends on the previous value(s) of a stochastic term. In both cases, the number of previous values is referred to as the __order__.

If an auto-regresive model is appropriate, the time-series should show strong auto-correlations. That is, the current value of the time-series should be correlated to previous values. This is precisely what I see in the data (representative time-series with linear interpolation, blue curves represent 95% confidence interval for an uncorrelated process):

![Representative auto-correlation functions](images/autocorr.png)

Other interpolation methods give similar results. Thus, we have an auto-regressive model, with no moving average component (__q__=0). The __order__ of the auto-regressive component can be found from partial auto-correlation plots, which gives the correlation which cannot be accounted for by previous lags. Thus, the lag at which the partial correlation function becomes zero gives the order of the AR process.

![Representative partial auto-correlation functions](images/pautocorr.png)

These results suggest the time-series is auto-regressive with order 1 (__p__=1).

### Summary of the AR Model

I will be fitting an ARIMA(1, 1, 0) model to the Trout Bog epilimnion time-series. What does the model look like?

A 1st order AR model has the form:

$y_t = c + \phi y_{t-1} + \epsilon_t$ 

where $t$ is time, $y$ is the time-series value, $c$ is a constant, $\phi$ is the autoregression parameter, and $\epsilon_t$ is the error term.

Our model will need to be first-differenced. That is, values of $y$ are replaced with their first differences, $y'$: 

$y'_t = y_t - y_{t-1}$

Performing a substitution and rearranging, we obtain:

$y_t = c + y_{t-1} + \phi(y_{t-1} + y_{t-2}) + \epsilon_t$

Note that some $y_t$ are known from data, and the others are parameters to be identified via the optimization.

## Parameter Optimization

As described above, my parameter estimation routine will have two features:

1. Rather than performing interpolation before the parameter estimation, I will perform interpolation simultaneously with the estimation. This allows the values of missing data points to emerge naturally from the model rather than assuming a fixed structure for the missing data (linear interpolation, etc). This also has the advantage of capturing dynamics that don't conform to the standard interpolation functions (such as blooms and other non-monotonic behavior).

2. For datasets with large gaps (such as winter), rather than interpolating across the long gaps, I will perform a single optimization over multiple time-series (of one year each) in parallel, rather than over a single time-series spanning the entire length of the dataset. The difference between these two formulations is shown conceptually below:

![Single vs multiple time-series](images/two-vs-one-time-series.png)

The performance of my routine will be benchmarked against routines which perform interpolation across a single time-series spanning the length of the dataset.

Parameters will be estimated using least squares estimation (as opposed to maximum likelihood). Apart from personal familiarity, I don't have a good justification for this choice and am open to changing my mind.

### Model Formulation for the Single OTU AR Model

For a single time-series, the nonlinear least squares formulation is:

\begin{aligned}
& \text{min} && \sum_{t \in T_{obs}} (y_t - y_{t,obs}) \\
& \text{s.t} && y_t = c + y_{t-1} + \phi(y_{t-1} + y_{t-2}) + \epsilon_t && t \in T_{obs} 
\end{aligned}

where $t$ is time, $y$ is the time-series value, $c$ is a constant, $\phi$ is the autoregression parameter, and $\epsilon_t$ is the error term. $T_{obs}$ is the set of time-points for which data were observed, and $y_{t,obs}$ is the observed value at time $t$. Thus, the least squares estimation only considers those time-points for which we have observed data.

This formulation is a non-linear program (NLP), which can be solved via freely-available, commercial-grade solvers like [IPOPT](https://projects.coin-or.org/Ipopt). NLPs require an initial parameter estimation to obtain a good solution, I'm still thinking about a heuristic for obtaining a good starting point.

For multiple time-series, the formulation is as follows:

\begin{aligned}
& \text{min} && \sum_{l \in L} \sum_{t \in T_{obs}} (y_t - y_{t,obs}) \\
& \text{s.t} && y_{l,t} = c + y_{l,t-1} + \phi(y_{l,t-1} + y_{l,t-2}) + \epsilon_{l,t} && l \in L && t \in T_{obs}  
\end{aligned}

where $l$ is the year and $L$ is the set of all years for which data are avaiable. Note that in both cases, there is still a single set of parameters (c, $\phi$) to be estimated. 

These optimization problems will be solved for each OTU independently. In later formulations, I will introduce vector auto-regression, in which I solve for interactions between OTUs.

## First Extended Model: Multivariate ARIMA Model

The second model I plan to develop is a multivariate auto-regressive model, in which the abundance of an OTU depends on its previous abundance as well as the previous abundance of other OTUs.

If there __n__ interacting species, then the multivariate ARIMA(1,1,0) model (VARIMA(1,1,0)) looks as follows:

$Y_t = C + Y_{t-1} + \Phi(Y_{t-1} + Y_{t-2}) + E_t$

where $Y_t$ is an $n \times 1$ vector of relative abundances, $C$ is a $n \times 1$ vector of constants, $\Phi$ is an $n \times n$ matrix whose elements $\phi_{i,j}$ give the effect of the abundance of species $j$ on the growth rate of species $i$, and $E_t$ is a $n \times 1$ vector of errors.

For a single time-series, the nonlinear least squares formulation is:

\begin{aligned}
& \text{min} && \sum_{n} \sum_{t \in T_{obs}} (y_{n,t} - y_{n,t_{obs}}) \\
& \text{s.t} && Y_t = C + Y_{t-1} + \Phi(Y_{t-1} + Y_{t-2}) + E_t && t \in T_{obs} 
\end{aligned}

This formulation is also non-linear program (NLP) and will be solved in a similar way as the uni-variate model. For multiple time-series, the formulation is as follows:

\begin{aligned}
& \text{min} && \sum_{l \in L} \sum_{n} \sum_{t \in T_{obs}} (y_{n,t} - y_{n,t_{obs}}) \\
& \text{s.t} && Y_{l,t} = C + Y_{l,t-1} + \Phi(Y_{l,t-1} + Y_{l,t-2}) + E_{l,t} && l \in L && t \in T_{obs} 
\end{aligned}

where $l$ is the year and $L$ is the set of all years for which data are avaiable. Note that in both cases, there is still a single set of parameters (C, $\Phi$) to be estimated.

## Second Extended Model: ARIMA Model with Environmental Terms

The third model I plan to develop is a univariate auto-regressive model, in which the abundance of an OTU depends on its previous abundance(s), the value of environmental parameters (covariates), and an error term. The parameter optimation problem is:

\begin{aligned}
& \text{min} && \sum_{l \in L} \sum_{t \in T_{obs}} (y_t - y_{t,obs}) \\
& \text{s.t} && y_{l,t} = c + y_{l,t-1} + \alpha u_t + \phi(y_{l,t-1} + y_{l,t-2}) + \epsilon_{l,t} && l \in L && t \in T_{obs}  
\end{aligned}

where $t$ is time, $y$ is the time-series value, $c$ is a constant, $\phi$ is the autoregression parameter, and $\epsilon_t$ is the error term. $u$ is a $q \times 1$ vector containing values of environmental variables, and $\alpha$ is an $1 \times q$ vector giving the strength of each environmental variable on the OTU abundance. Again, $l$ is the year and $L$ is the set of all years for which data are available. The parameters to be estimated are (c, $\alpha$, $\phi$).

## Third Extended Model: VARIMA Model with Environmental Terms

The final model I plan to develop is a multivariate auto-regressive model, in which the abundance of an OTU depends on its previous abundance(s), the value of environmental parameters (covariates), and an error term. The problem generalizes in a similar way:

\begin{aligned}
& \text{min} && \sum_{l \in L} \sum_{n} \sum_{t \in T_{obs}} (y_{n,t} - y_{n,t_{obs}}) \\
& \text{s.t} && Y_{l,t} = C + Y_{l,t-1} + AU_t + \Phi(y_{l,t-1} + Y_{l,t-2}) + E_{l,t} && l \in L && t \in T_{obs}  
\end{aligned}

where $t$ is time, $Y_t$ is an $n \times 1$ vector of relative abundances, $C$ is a $n \times 1$ vector of constants,
$U$ is a $q \times 1$ vector containing values of environmental variables, and A is an $n \times q$ matrix whose elements $\alpha_{i,j}$ give the strength of each environmental variable on the OTU abundance. $\Phi$ is an $n \times n$ matrix whose elements $\phi_{i,j}$ give the effect of the abundance of species $j$ on the growth rate of species $i$, and $E_t$ is a $n \times 1$ vector of errors. The parameters to be estimated are (C, A, and $\Phi$).

## Model Validation

The final section will focus on validating the models, including obtaining confidence intervals for model parameters and performing cross-validation to test model accuracy. 

### Confidence Intervals
The confidence interval of a parameter is the range of values (the interval) that acts as a good estimate of the unknown parameter. A good optimization routine should result in narrow confidence intervals for most parameters, and parameters with narrow intervals are suggestive of ecological interactions. I anticipate that not all parameters will have good (narrow) confidence intervals. Parameters with poor (wide) either cannot be determined from the data, or are unimportant in the model. I am presently undecided about the best way to obtain confidence intervals. For multi-variate models, parameters include interaction coefficients between taxa and the environment, and may point to ecological interactions among taxa.

### Cross-validation
Cross-validation is a technique for assessing how well a model generalizes to data it hasn't seen before.

I propose to cross-validate my models using leave-p-out validation on time-series. That is, I will remove 'p' years of data and build the model (the training set), and assess the model's ability to predict the remaining years of data (the validaion set). Accuracy will be evaluated using mean squared error. By performing the validation for different values of 'p', I will also understand how many years of data are required for robust parameter estimation.

Leave-p-out validation is "exhaustive," in that it assesses all possible ways to partition the data into testing and training sets. If my optimization routine proves computationally expensive, I will perform non-exhaustive cross-validation.

It __may__ be possible to perform leave-p-out validation on individual samples as well. While this is unlikely to be feasible given the large number of datapoints, the results will suggest the number of samples required for robust parameter estimation.

### Comparing Multiple Models
I anticipate that the more complex, multivariate models will provide a better fit to the data, as described by the residual squared error. This value is the objective function for the parameter estimation problem so is easily obtainable.